In [1]:
import numpy as np
 
def create_matrices(Tuple, values, alpha, c, p, M, N):
  m = M // c + 1*(M % c > 0)
  n = N // p + 1*(N % p > 0)
  i = Tuple[0]
  j = Tuple[1]
 
  if i < (c-1):
    x1 = m
  else:
    x1 = M - m*(c-1)
    
  if j < (p-1):
    x2 = n
  else:
    x2 = N - n*(p-1)
  
  m1 = np.zeros([x1, x2], dtype=np.int64)
  y1 = [t[0] for t in values]
  y2 = [t[1] for t in  values]
  m1[y1, y2] = 1
  C = 1 + m1*alpha
  return (((Tuple), (m1, C)))
   
  
#create m1 and C matrices
def create_m1_C(data, c, p, M, N, alpha):
  numpartitions = c*p
  m1 = M // c + 1*(M % c > 0)
  n1 = N // p + 1*(N % p > 0)
  entries = data.map(lambda x: ((x[0] // m1, x[1] // n1), (x[0] % m1, x[1] % n1 ))).partitionBy(numpartitions, lambda x: x[0]*p + x[1])
  return  (entries.groupByKey().map(lambda x :create_matrices(x[0], x[1], alpha, c, p, M, N )))
 

#placeholder for splitting customers and products. 
#use Lagrangian multipliers for the general case of N being as large as M
def lang_opt(k, M, N):
  return ((k // 3  , 3))

#load purchase data to create the RDD of m1 and C submatrices.
#alpha is weighting parameter, f is rank (no of latent factors to fit)
#numpartitions is number of partitions to divide data into 

#def load_ratings(alpha, numpartitions, f, Lambda):
def load_ratings(alpha, numpartitions):
  data = sc.textFile("/FileStore/tables/mf8ppr9o1486658249082/Customer_Purchases3.csv")
  header = data.first() #extract header
  data = data.filter(lambda row : row != header) 
  data = data.map(lambda l: l.split(",")).map(lambda x: (int(x[0]) -1, int(x[1]) -1))
  
  N = data.map(lambda x: x[1]).top(1)[0] + 1
  M = data.map(lambda x: x[0]).top(1)[0] + 1
  partitions = lang_opt(numpartitions, M, N)
  c = partitions[0]
  p = partitions[1]
  m = M // c + 1*(M % c > 0)
  n = N // p + 1*(N % p > 0)
  matrices = create_m1_C(data, c, p, M, N, alpha).persist()
  return ([matrices, [c,p,M,N, m, n]])
  
def flatten2(x):
  p = partition_params.value[1]
  m = partition_params.value[4]
  return ([((int(x[0]-1) // m, i),  (int(x[0]-1) % m, x[1:])) for i in range(p)])

def flatten3(x):
  c = partition_params.value[0]
  n = partition_params.value[5]
  return ([((i,int(x[0]-1) // n),  (int(x[0]-1) % n, x[1:])) for i in range(c)])

  

def create_vecs(values):
  sorted_values = sorted(values, key = (lambda x: x[0]))
  vecs = [t[1] for t in  sorted_values]
  return (np.array(vecs))

#Load customer features as per the existing partition
def load_customer_features(matrices):
  customer_data = sc.textFile("/FileStore/tables/f11biiqi1486887764969/Customer_Features.csv")
  header = customer_data.first() #extract header
  customer_data = customer_data.filter(lambda row : row != header) 
  customer_data = customer_data.map(lambda l: l.split(",")).map(lambda x: map(float, x))
  customer_data = customer_data.flatMap(lambda  x: flatten2(x))
  a = customer_data.groupByKey().mapValues(create_vecs)
  a = matrices.join(a).mapValues(lambda x: x[1]).persist()
  cust_len = a.lookup((0,0))[0].shape[1]
  return ([a, cust_len])

#Load customer features as per the existing partition
def load_product_features(matrices):
  product_data = sc.textFile("/FileStore/tables/zqjp4ejm1486897287081/Product_Features.csv")
  header = product_data.first() #extract header
  product_data = product_data.filter(lambda row : row != header) 
  product_data = product_data.map(lambda l: l.split(",")).map(lambda x: map(float, x))
  product_data = product_data.flatMap(lambda  x: flatten3(x))
  b = product_data.groupByKey().mapValues(create_vecs)
  b = matrices.join(b).mapValues(lambda x: x[1]).persist()
  prod_len = b.lookup((0,0))[0].shape[1]
  return ([b, prod_len])

  
  

In [2]:
def generate_factors(vals):
  c = vals[0]
  m = vals[1]
  M = vals[2]
  i = vals[3]
  f = vals[4] + vals[5]
  if i < (c-1):
    x = m
  else:
    x = M - m*(c-1)
  return ((i, np.random.randn(x,f)))


  
def propagate_customer_factors(matrices, y):
  p = partition_params.value[1]
  y = sc.parallelize(y).flatMap(lambda x: [((x[0], i), (x[1])) for i in range(p)])
  return (matrices.join(y).mapValues(lambda x: x[1]))

def propagate_product_factors(matrices, y):
  c = partition_params.value[0]
  y = sc.parallelize(y).flatMap(lambda x: [((i, x[0]), (x[1])) for i in range(c)])
  return (matrices.join(y).mapValues(lambda x: x[1]))
  
def initialize_customer_factors(matrices):
  c = partition_params.value[0]
  M = partition_params.value[2]
  m = partition_params.value[4]
  f_ = f.value
  vals  = [(c, m, M, i, f_, prod_len.value) for i in range(c)]
  y = list(map(lambda x: generate_factors(x), vals))
  return (propagate_customer_factors(matrices, y))

  
  
def initialize_product_factors(matrices):
  p = partition_params.value[1]
  N = partition_params.value[3]
  n = partition_params.value[5]
  f_ = f.value
  vals  = [(p, n, N, i, f_, cust_len.value) for i in range(p)]
  y = list(map(lambda x: generate_factors(x), vals))
  return (propagate_product_factors(matrices, y))


  
  

In [3]:
from numpy.linalg import inv

def update_customer_messages(matrices, yblock, a, b):
  y = yblock.mapValues(lambda x: x[:, :f.value])
  yy = yblock.mapValues(lambda x: x[:, f.value:])
  y_aug = y.join(yy).join(b)
  msg = matrices.mapValues(lambda x: np.hstack((x[0], x[1]))).join(a).join(y_aug).mapValues(lambda x : update(x, cust_len.value))
  return msg
  
def update(values, dim):
  #extract  relavent values
  y = values[1][0][0]
  b = values[1][1]
  y_o = np.hstack((y,b))
  x1 = values[0][0]
  yy = values[1][0][1]
  a = values[0][1]
  x2 = np.hstack((x1,a))
  return np.apply_along_axis(lambda x: update_msg_axis(x, y_o, yy, dim), 1, x2)

def update_msg_axis(vector, y_o,  yy, dim):
  d = (len(vector) - dim) // 2 #this is the same as f
  Filter = vector[d : 2*d]
  p_u = vector[:d]
  p_u = p_u[Filter > -1]
  n_adj = p_u.size
  temp = Filter[Filter > -1]
  c_u = temp*np.eye(n_adj) 
  
  a_u = vector[-dim:]
  yy_adj = yy[Filter > -1, :]
  pu_corr =np.dot(yy_adj, a_u) 
  p_u = p_u - pu_corr
  y_o_filter = y_o[Filter > -1,:]

  #update all the messages
  msg1 = np.dot(y_o_filter.T, np.dot(c_u, p_u)).T
  msg2 = np.dot(y_o_filter.T, np.dot(c_u, y_o_filter)).ravel()
  #retunrs the flatenned matrices of dimenion 1 X [f*f +f]
  return (np.hstack((msg2, msg1)))

def update_customerfactors(x):
  return np.apply_along_axis(lambda vec: compute_factors(vec, prod_len.value), 1, x)
                                                              
def compute_factors(vec, dim):
  f_ = dim + f.value
  Lambda_ = Lambda.value
  x1 = vec[:-f_]
  
  x1 = x1.reshape(f_,-1)
  x2 = vec[-f_:]
 
  temp = inv(x1 + Lambda_*np.eye(f_))
  val = np.dot(temp,x2.T).T
  return val




In [4]:
def reduce_propagate_customer_messages(msgs):
  c = partition_params.value[0]
  vals  = range(c)
  y = list(map(lambda x: reduce_cust_msg(x, msgs), vals))
  return (propagate_customer_factors(matrices, y))


def reduce_cust_msg(vals, msgs):
  #use i to filter msgs and then reduce to get final value which is passed
  msgs_reduced = msgs.filter(lambda x: x[0][0] == vals).map(lambda x: x[1]).reduce(lambda x,y: x + y)
  factors = update_customerfactors(msgs_reduced)
  return ((vals, factors))

def CustomerFactor_Update(msgs):
  p = partition_params.value[1] 
  if p < 2:
    return msgs.mapValues(lambda x: update_customerfactors(x))
  else:
    return reduce_propagate_customer_messages(msgs)


In [5]:
from numpy.linalg import inv

def update_product_messages(matrices, xblock, a, b):
  x = xblock.mapValues(lambda x: x[:, :f.value])
  xx = xblock.mapValues(lambda x: x[:, f.value:])
  x_aug = x.join(xx).join(a)
  msg = matrices.mapValues(lambda x: np.vstack((x[0], x[1]))).mapValues(lambda x :  x.T).join(b).join(x_aug).mapValues(lambda x : update(x, prod_len.value))
  return msg
  


def update_productfactors(x):
  return np.apply_along_axis(lambda vec: compute_factors(vec, cust_len.value), 1, x)
                                                              



In [6]:
def reduce_propagate_product_messages(msgs):
  p = partition_params.value[1]
  vals  = range(p)
  y = list(map(lambda x: reduce_prod_msg(x, msgs), vals))
  return (propagate_product_factors(matrices, y))


def reduce_prod_msg(vals, msgs):
  #use i to filter msgs and then reduce to get final value which is passed
  msgs_reduced = msgs.filter(lambda x: x[0][1] == vals).map(lambda x: x[1]).reduce(lambda x,y: x + y)
  factors = update_productfactors(msgs_reduced)
  return ((vals, factors))

def ProductFactor_Update(msgs):
  c = partition_params.value[0] 
  if c < 2:
    return msgs.mapValues(lambda x: update_productfactors(x))
  else:
    return reduce_propagate_product_messages(msgs)

In [7]:
def model_training(matrices, a, b, sweeps):
  #f = sc.broadcast(f_)
  #Lambda = sc.broadcast(Lambda_)
  
  #inititalize, propagate the customer and product factors
  xblock = initialize_customer_factors(matrices).persist()
  yblock = initialize_product_factors(matrices).persist()
  
  for k in range(sweeps):
    msg = '%s %d' % ('Starting iteration #: ', k)  
    print msg
        
    # update customer factors
    msgs = update_customer_messages(matrices, yblock, a, b).persist()
    xblock = CustomerFactor_Update(msgs).persist()
    print xblock.count()
    msg = 'Completed update of customer factors' 
    print msg
    
    # update product factors
    msgs = update_product_messages(matrices, xblock, a, b).persist()
    yblock = ProductFactor_Update(msgs).persist()
    print yblock.count()
    msg = 'Completed update of product factors' 
    print msg
  
  return ([xblock, yblock])
  
  
  
  

In [8]:
#split into test-train
#some of the purchases are marked fore testing and removed from training
#C[i,j] == -2 indicates that that that data-point has been flagged for testing
#sample is number of purchases per customer that must be flagged for testing
#k is threshold parameter; if customer has < k purchases, then none of his purchases flagged for testing
#use this if model is evaluated for AUC
def data_test_train(matrices, fraction):
  return matrices.mapValues(lambda x: test_train_split(x, fraction))

def test_train_split(x, fraction):
    m1 = x[0]
    C = x[1]
    x1 = np.hstack((m1, C))
    mat =  np.apply_along_axis(lambda vector: remove_purchases_axis(vector, fraction), 1, x1)
    d = mat.shape[1] // 2
    return (mat[:, :d], mat[:, d:])
  
def remove_purchases_axis(vector, fraction):
    d = len(vector) // 2
    tmp_m1  = vector[:d]
    C = vector[d:]
    s = np.sum(tmp_m1)
    sample = int(s*fraction)
    if (sample > 0):
      shape = tmp_m1.shape
      p = np.full(shape,1.0)
      p[tmp_m1 == 0] = 0
      valid = tmp_m1[tmp_m1 > 0].size
      p = p/valid
      inds = np.random.choice(tmp_m1.size, sample, replace=False, p = p)
      C[inds] = -2
    return np.concatenate((tmp_m1, C), axis=1)
      




        

    
    


  

In [9]:
#load the ratings data to create the matrices RDD
[matrices, partition_params] = load_ratings(40, 36)
partition_params = sc.broadcast(partition_params) #broadcast partition_params to all worker nodes


[a, cust_len] = load_customer_features(matrices)
print cust_len
cust_len = sc.broadcast(cust_len)
[b, prod_len] = load_product_features(matrices)
print prod_len
prod_len = sc.broadcast(prod_len)

matrices = data_test_train(matrices, 0.5).persist()
f = sc.broadcast(10)
Lambda = sc.broadcast(10)
[xblock, yblock] = model_training(matrices, a, b, 10)
print xblock.lookup((0,0))[0].shape
print xblock.lookup((0,2))[0]

print yblock.lookup((0,0))[0].shape
print yblock.lookup((0,2))[0]


In [10]:
def predictions(a, b, xblock, yblock):
  x = xblock.mapValues(lambda x: x[:, :f.value])
  xx = xblock.mapValues(lambda x: x[:, f.value:])
  XX_ = x.join(a).mapValues(lambda x: np.hstack((x[0], x[1])))
  XX = XX_.join(xx).mapValues(lambda x: np.hstack((x[0], x[1])))
  YY = yblock.join(b).mapValues(lambda x: np.hstack((x[0], x[1])))
  return (XX.join(YY).mapValues(lambda x: np.dot(x[0], x[1].T)))
  
 


In [11]:
def modeleval_AUC(pred1, matrices):
  c = partition_params.value[0]
  vals  = range(c)
  y = list(map(lambda x: reduce_along_p(x, pred1, matrices), vals))
  y = sc.parallelize(y)
  y = y.map(lambda x: arbit(x)).map(lambda x: np.array(x))
  final = y.reduce(lambda x, y: x+ y)
  #ind = y[:,-1]
  ##inds = sum(ind > 0)
  #yy = y[:, 0:2]
  #final = np.apply_along_axis(lambda x: x[0]/x[1], 1, yy)
  return final[0]/final[1]
  #return final

def eval(x):
  m1 = x[0][0]
  C = x[0][1]
  pred1 = x[1]
  x1 = np.hstack((np.hstack((m1,C)), pred1))
  return  np.apply_along_axis(lambda vector: eval_axis(vector), 1, x1)
  
def arbit(x):
  kk1 = np.array(x)
 
  dim = kk1.shape[1]
  kk2 = kk1.reshape(dim,-1)
  
  ind = kk2[:,-1]
  inds = sum(ind > 0)
  yy = kk2[:, 0:2]
  final = np.sum(np.apply_along_axis(lambda x: kaka(x), 1, yy))
  return np.array([final, inds])

def kaka(x):
  if x[1] > 0:
    return x[0]/x[1]
  else:
    return 0
  
def eval_axis(vector):
  d = len(vector) // 3
  num =0
  m1 = vector[:d]
  C = vector[d:2*d]
  pred2 = vector[2*d:]
  preds = pred2[(m1 == 0) & (C > 0)]
  testscore = pred2[C == -2]
  l = testscore.size
  result1 = 0
  result2 = 0
  count = 0
  if ((l > 0) & (preds.size > 0)):
    for j in range(0,l):
                temp = testscore[j] - preds
                num = num + sum(temp > 0)
    result1 = float(num)
    result2 = float(preds.size*l)
    count = count +1
    
  return (result1, result2, count)



  


def reduce_along_p(vals, pred1, matrices):
  #use i to filter msgs and then reduce to get final value which is passed
  pred_ = pred1.filter(lambda x: x[0][0] == vals)
  matrices_ = matrices.filter(lambda x: x[0][0] == vals)
  
  msgs_reduced = matrices_.join(pred_).mapValues(lambda x: eval(x))
  final = msgs_reduced.map(lambda x: x[1]).reduce(lambda x, y: x + y)
  
  return final
  
  

  

In [12]:
pred1 = predictions(a, b, xblock, yblock).persist()
AUC = modeleval_AUC(pred1, matrices)
print AUC

